# Day 02 — Baseline model (Logistic Regression)

In this notebook we build a **baseline classifier**. A baseline gives you a reference point so you can tell if future models are actually better.

We will cover:
- What a baseline model is
- Train/test splitting
- Logistic regression for binary classification
- Core evaluation metrics (accuracy, precision, recall, F1, ROC-AUC)
- Simple threshold tuning


## 1) Create a small classification dataset
We’ll reuse a tiny, interpretable dataset about students and exam outcomes. In real projects, you would load data from a file.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

students = pd.DataFrame(
    {
        "study_hours": [1, 2, 3, 4, 5, 2, 3, 4, 6, 7, 1, 5, 8, 2, 6],
        "practice_tests": [0, 1, 1, 2, 2, 0, 1, 2, 3, 3, 0, 2, 4, 1, 3],
        "passed": [0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
    }
)

students.head()


## 2) Split into train and test
We need a **hold-out test set** to evaluate generalization. We'll use 80% for training and 20% for testing.


In [ ]:
X = students[["study_hours", "practice_tests"]]
y = students["passed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


## 3) Train a baseline model
Logistic regression is a strong baseline for binary classification.
We use a scaler because features are on different scales.


In [ ]:
model = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression()),
    ]
)

model.fit(X_train, y_train)


## 4) Evaluate with standard metrics
A single metric can hide important behavior. We’ll look at multiple metrics.


In [ ]:
preds = model.predict(X_test)
probs = model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds))
print("Recall:", recall_score(y_test, preds))
print("F1:", f1_score(y_test, preds))
print("ROC-AUC:", roc_auc_score(y_test, probs))

print("\nConfusion matrix:\n", confusion_matrix(y_test, preds))
print("\nClassification report:\n", classification_report(y_test, preds))


## 5) Threshold tuning
By default, logistic regression uses a 0.5 probability threshold. You can change it to trade off precision and recall.


In [ ]:
threshold = 0.4
custom_preds = (probs >= threshold).astype(int)

print("Custom threshold:", threshold)
print("Precision:", precision_score(y_test, custom_preds))
print("Recall:", recall_score(y_test, custom_preds))


## 6) What to do next
Now that you have a baseline, the next steps are:
- Try a more flexible model (Decision Trees in Day 03)
- Compare metrics to ensure real improvement
- Keep the baseline as a reference
